In [ ]:
实战_Requests库 + 正则表达式：抓取猫眼-top100电影
2019.03.13

第一步：获取 单页索引页



In [1]:
import requests
from requests.exceptions import RequestException
import re

def get_one_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    print(html)

if __name__ == '__main__':
    main()

<!DOCTYPE html>

<!--[if IE 8]><html class="ie8"><![endif]-->
<!--[if IE 9]><html class="ie9"><![endif]-->
<!--[if gt IE 9]><!--><html><!--<![endif]-->
<head>
  <title>TOP100榜 - 猫眼电影 - 一网打尽好电影</title>
  
  <link rel="dns-prefetch" href="//p0.meituan.net"  />
  <link rel="dns-prefetch" href="//p1.meituan.net"  />
  <link rel="dns-prefetch" href="//ms0.meituan.net" />
  <link rel="dns-prefetch" href="//ms1.meituan.net" />
  <link rel="dns-prefetch" href="//analytics.meituan.com" />
  <link rel="dns-prefetch" href="//report.meituan.com" />
  <link rel="dns-prefetch" href="//frep.meituan.com" />

  
  <meta charset="utf-8">
  <meta name="keywords" content="猫眼电影,电影排行榜,热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100">
  <meta name="description" content="猫眼电影热门榜单,包括热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100,多维度为用户进行选片决策">
  <meta http-equiv="cleartype" content="yes" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
  <meta name="renderer" content="webkit" />

  <meta name="HandheldFriendly" content="true" 

In [ ]:


第二步，对网页进行解析，采用 def parse_one_page(url)

获取到元组信息


In [1]:
import requests
from requests.exceptions import RequestException
import re

def get_one_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

# 第二步，插入的解析页,解析html代码

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         +'.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         +'.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
    print(items)
    
def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    parse_one_page(html)

if __name__ == '__main__':
    main()

[('1', 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', '霸王别姬', '\n                主演：张国荣,张丰毅,巩俐\n        ', '上映时间：1993-01-01(中国香港)', '9.', '6'), ('2', 'http://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', '肖申克的救赎', '\n                主演：蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿\n        ', '上映时间：1994-10-14(美国)', '9.', '5'), ('3', 'http://p0.meituan.net/movie/54617769d96807e4d81804284ffe2a27239007.jpg@160w_220h_1e_1c', '罗马假日', '\n                主演：格利高里·派克,奥黛丽·赫本,埃迪·艾伯特\n        ', '上映时间：1953-09-02(美国)', '9.', '1'), ('4', 'http://p0.meituan.net/movie/e55ec5d18ccc83ba7db68caae54f165f95924.jpg@160w_220h_1e_1c', '这个杀手不太冷', '\n                主演：让·雷诺,加里·奥德曼,娜塔莉·波特曼\n        ', '上映时间：1994-09-14(法国)', '9.', '5'), ('5', 'http://p1.meituan.net/movie/f5a924f362f050881f2b8f82e852747c118515.jpg@160w_220h_1e_1c', '教父', '\n                主演：马龙·白兰度,阿尔·帕西诺,詹姆斯·肯恩\n        ', '上映时间：1972-03-24(美国)', '9.', '3'), ('6', 'http://p1.meituan.net/movie/06

In [ ]:



第三步，针对以上杂乱的信息进行格式化 提取 

效果：提取 10个 键值对 形式



In [4]:
import requests
from requests.exceptions import RequestException
import re

def get_one_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

# 第二步，插入的解析页,解析html代码

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         +'.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         +'.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)

#第三步，将其变成字典的形式
    for item in items:
        yield {
            'index':item[0],
            'image':item[1],
            'title':item[2],
            'actor':item[3].strip()[3:], # 使用切片-strip功能，去掉 主演：这三个字符
            'time':item[4].strip()[5:],
            'score':item[5]+item[6]
        }

    
def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    for item in parse_one_page(html): # 第三步，提取生成器，打印效果
        print(item)

if __name__ == '__main__':
    main()

{'index': '1', 'image': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01(中国香港)', 'score': '9.6'}
{'index': '2', 'image': 'http://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-10-14(美国)', 'score': '9.5'}
{'index': '3', 'image': 'http://p0.meituan.net/movie/54617769d96807e4d81804284ffe2a27239007.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
{'index': '4', 'image': 'http://p0.meituan.net/movie/e55ec5d18ccc83ba7db68caae54f165f95924.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
{'index': '5', 'image': 'http://p1.meituan.net/movie/f5a924f362f050881f2b8f82e852747c118515.jpg@160w_220h_1e_1c', 'title': '教父', 'actor': '马龙·白兰度,阿尔·帕西诺,詹姆斯·肯恩', 'time': '1972-03-24(

In [ ]:


第四步：将其存储到文件中
    def write_to_file(content):


In [6]:
import requests
from requests.exceptions import RequestException
import re
import json

def get_one_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

# 第二步，插入的解析页,解析html代码

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         +'.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         +'.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)

#第三步，将其变成字典的形式
    for item in items:
        yield {
            'index':item[0],
            'image':item[1],
            'title':item[2],
            'actor':item[3].strip()[3:], # 使用切片-strip功能，去掉 主演：这三个字符
            'time':item[4].strip()[5:],
            'score':item[5]+item[6]
        }

def write_to_file(content): # 第四步，存储到文件
    with open('result.txt','a') as f:
        f.write(json.dumps(content) + '\n') # 使用json.dump，把其字典形式 转为 字符串，再添加个换行
        f.close()
    
def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    for item in parse_one_page(html): # 第三步，提取生成器，打印效果
        print(item)
        write_to_file(item) # 第四步，将其传入到文件

if __name__ == '__main__':
    main()

{'index': '1', 'image': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01(中国香港)', 'score': '9.6'}
{'index': '2', 'image': 'http://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-10-14(美国)', 'score': '9.5'}
{'index': '3', 'image': 'http://p0.meituan.net/movie/54617769d96807e4d81804284ffe2a27239007.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
{'index': '4', 'image': 'http://p0.meituan.net/movie/e55ec5d18ccc83ba7db68caae54f165f95924.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
{'index': '5', 'image': 'http://p1.meituan.net/movie/f5a924f362f050881f2b8f82e852747c118515.jpg@160w_220h_1e_1c', 'title': '教父', 'actor': '马龙·白兰度,阿尔·帕西诺,詹姆斯·肯恩', 'time': '1972-03-24(

In [ ]:

第四步：维修，将 result.txt中 unix编码转为 中文形式
            在 write_to_file 中，加入 encoding='utf-8'  和 ‘ensure_ascii=False’
    

In [7]:
import requests
from requests.exceptions import RequestException
import re
import json

def get_one_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

# 第二步，插入的解析页,解析html代码

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         +'.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         +'.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)

#第三步，将其变成字典的形式
    for item in items:
        yield {
            'index':item[0],
            'image':item[1],
            'title':item[2],
            'actor':item[3].strip()[3:], # 使用切片-strip功能，去掉 主演：这三个字符
            'time':item[4].strip()[5:],
            'score':item[5]+item[6]
        }

def write_to_file(content): # 第四步，存储到文件
    with open('result.txt','a',encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n') # 使用json.dump，把其字典形式 转为 字符串，再添加个换行
        f.close()
    
def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    for item in parse_one_page(html): # 第三步，提取生成器，打印效果
        print(item)
        write_to_file(item) # 第四步，将其传入到文件

if __name__ == '__main__':
    main()

{'index': '1', 'image': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01(中国香港)', 'score': '9.6'}
{'index': '2', 'image': 'http://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-10-14(美国)', 'score': '9.5'}
{'index': '3', 'image': 'http://p0.meituan.net/movie/54617769d96807e4d81804284ffe2a27239007.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
{'index': '4', 'image': 'http://p0.meituan.net/movie/e55ec5d18ccc83ba7db68caae54f165f95924.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
{'index': '5', 'image': 'http://p1.meituan.net/movie/f5a924f362f050881f2b8f82e852747c118515.jpg@160w_220h_1e_1c', 'title': '教父', 'actor': '马龙·白兰度,阿尔·帕西诺,詹姆斯·肯恩', 'time': '1972-03-24(

In [ ]:

第五步：循环，将10页的内容都抓取过来
1、先看网页结构
2、找到 offset


In [12]:
import requests
from requests.exceptions import RequestException
import re
import json

def get_one_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

# 第二步，插入的解析页,解析html代码

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         +'.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         +'.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)

#第三步，将其变成字典的形式
    for item in items:
        yield {
            'index':item[0],
            'image':item[1],
            'title':item[2],
            'actor':item[3].strip()[3:], # 使用切片-strip功能，去掉 主演：这三个字符
            'time':item[4].strip()[5:],
            'score':item[5]+item[6]
        }

def write_to_file(content): # 第四步，存储到文件
    with open('result100.txt','a',encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n') # 使用json.dump，把其字典形式 转为 字符串，再添加个换行
        f.close()
    
def main(offset): # 第五步，加入 offset
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html): # 第三步，提取生成器，打印效果
        print(item)
        write_to_file(item) # 第四步，将其传入到文件

if __name__ == '__main__':
    for i in range(10):
        main(i*10)    

{'index': '1', 'image': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01(中国香港)', 'score': '9.6'}
{'index': '2', 'image': 'http://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-10-14(美国)', 'score': '9.5'}
{'index': '3', 'image': 'http://p0.meituan.net/movie/54617769d96807e4d81804284ffe2a27239007.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
{'index': '4', 'image': 'http://p0.meituan.net/movie/e55ec5d18ccc83ba7db68caae54f165f95924.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
{'index': '5', 'image': 'http://p1.meituan.net/movie/f5a924f362f050881f2b8f82e852747c118515.jpg@160w_220h_1e_1c', 'title': '教父', 'actor': '马龙·白兰度,阿尔·帕西诺,詹姆斯·肯恩', 'time': '1972-03-24(

{'index': '51', 'image': 'http://p0.meituan.net/movie/85c2bfba6025bfbfb53291ae5924c215308805.jpg@160w_220h_1e_1c', 'title': '神偷奶爸', 'actor': '史蒂夫·卡瑞尔,杰森·席格尔,拉塞尔·布兰德', 'time': '2010-07-09(美国)', 'score': '9.0'}
{'index': '52', 'image': 'http://p0.meituan.net/movie/47dd790e19dad72b50580641de5608c5199014.jpg@160w_220h_1e_1c', 'title': '飞屋环游记', 'actor': '爱德华·阿斯纳,乔丹·长井,鲍勃·彼德森', 'time': '2009-08-04', 'score': '8.9'}
{'index': '53', 'image': 'http://p1.meituan.net/movie/5ca6ffcbb994a51cd6215e7c4fff2d9b71039.jpg@160w_220h_1e_1c', 'title': '黑客帝国3：矩阵革命', 'actor': '基努·里维斯,雨果·维文,凯瑞-安·莫斯', 'time': '2003-11-05', 'score': '8.8'}
{'index': '54', 'image': 'http://p0.meituan.net/movie/457a35fda360cb72090fa6dcbd1db3c1275333.jpg@160w_220h_1e_1c', 'title': '怦然心动', 'actor': '玛德琳·卡罗尔,卡兰·麦克奥利菲,艾丹·奎因', 'time': '2010-08-06(美国)', 'score': '8.9'}
{'index': '55', 'image': 'http://p0.meituan.net/movie/4bb144bc0a674ba6908349018fd092e6330929.jpg@160w_220h_1e_1c', 'title': '三傻大闹宝莱坞', 'actor': '阿米尔·汗,黄渤,卡琳娜·卡普尔', 'time'

{'index': '91', 'image': 'http://p1.meituan.net/movie/73349facab53529ab9e079c6c8c7c059281729.jpg@160w_220h_1e_1c', 'title': '七武士', 'actor': '三船敏郎,志村乔,千秋实', 'time': '1954-04-26(日本)', 'score': '9.1'}
{'index': '92', 'image': 'http://p1.meituan.net/movie/2c0a5fedf4b43d142121b91c6ccabe1b59051.jpg@160w_220h_1e_1c', 'title': '一一', 'actor': '吴念真,金燕玲,李凯莉', 'time': '2000-09-20(法国)', 'score': '8.9'}
{'index': '93', 'image': 'http://p1.meituan.net/movie/30310858fdab34c7a17cfd7ec8ad8bfc112201.jpg@160w_220h_1e_1c', 'title': '完美的世界', 'actor': '凯文·科斯特纳,克林特·伊斯特伍德,T·J·劳瑟', 'time': '1993-11-24(美国)', 'score': '8.9'}
{'index': '94', 'image': 'http://p0.meituan.net/movie/0018b57299d0d4540330a31244c880a9112971.jpg@160w_220h_1e_1c', 'title': '海洋', 'actor': '雅克·贝汉,姜文,兰斯洛特·贝汉', 'time': '2011-08-12', 'score': '9.0'}
{'index': '95', 'image': 'http://p1.meituan.net/movie/36a893c53a13f9bb934071b86ae3b5c492427.jpg@160w_220h_1e_1c', 'title': '爱·回家', 'actor': '俞承豪,金艺芬,童孝熙', 'time': '2002-04-05(韩国)', 'score': '9.0'}
{

In [ ]:


第六步：如果想实现 秒抓 的效果，需要引入一个 进程池，提供指定的进程，
        明显提高抓取效率


In [ ]:
import requests
from requests.exceptions import RequestException
from multiprocessing import Pool
import re
import json

def get_one_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

# 第二步，插入的解析页,解析html代码

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         +'.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         +'.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)

#第三步，将其变成字典的形式
    for item in items:
        yield {
            'index':item[0],
            'image':item[1],
            'title':item[2],
            'actor':item[3].strip()[3:], # 使用切片-strip功能，去掉 主演：这三个字符
            'time':item[4].strip()[5:],
            'score':item[5]+item[6]
        }

def write_to_file(content): # 第四步，存储到文件
    with open('result1010.txt','a',encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n') # 使用json.dump，把其字典形式 转为 字符串，再添加个换行
        f.close()
    
def main(offset): # 第五步，加入 offset
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html): # 第三步，提取生成器，打印效果
        print(item)
        write_to_file(item) # 第四步，将其传入到文件

if __name__ == '__main__':
    for i in range(10):
        main(i*10)    
    pool = Pool()
    pool.map(main, [i*10 for i in range(10)])

{'index': '1', 'image': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01(中国香港)', 'score': '9.6'}
{'index': '2', 'image': 'http://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-10-14(美国)', 'score': '9.5'}
{'index': '3', 'image': 'http://p0.meituan.net/movie/54617769d96807e4d81804284ffe2a27239007.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
{'index': '4', 'image': 'http://p0.meituan.net/movie/e55ec5d18ccc83ba7db68caae54f165f95924.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
{'index': '5', 'image': 'http://p1.meituan.net/movie/f5a924f362f050881f2b8f82e852747c118515.jpg@160w_220h_1e_1c', 'title': '教父', 'actor': '马龙·白兰度,阿尔·帕西诺,詹姆斯·肯恩', 'time': '1972-03-24(

{'index': '47', 'image': 'http://p0.meituan.net/movie/df15efd261060d3094a73ef679888d4f238149.jpg@160w_220h_1e_1c', 'title': '十二怒汉', 'actor': '亨利·方达,李·科布,马丁·鲍尔萨姆', 'time': '1957-04-13(美国)', 'score': '9.1'}
{'index': '48', 'image': 'http://p1.meituan.net/movie/1d0fa86bcf7a44484b9c16ac6af5be68191952.jpg@160w_220h_1e_1c', 'title': '速度与激情5', 'actor': '范·迪塞尔,保罗·沃克,道恩·强森', 'time': '2011-05-12', 'score': '9.2'}
{'index': '49', 'image': 'http://p1.meituan.net/movie/8194ae885ed9419aadf35c196af86ba4239039.jpg@160w_220h_1e_1c', 'title': '驯龙高手', 'actor': '杰伊·巴鲁切尔,杰拉德·巴特勒,亚美莉卡·费雷拉', 'time': '2010-05-14', 'score': '9.0'}
{'index': '50', 'image': 'http://p1.meituan.net/movie/f8e9d5a90224746d15dfdbd53d4fae3d209420.jpg@160w_220h_1e_1c', 'title': '勇敢的心', 'actor': '梅尔·吉布森,苏菲·玛索,帕特里克·麦高汉', 'time': '1995-05-24(美国)', 'score': '8.8'}
{'index': '51', 'image': 'http://p0.meituan.net/movie/85c2bfba6025bfbfb53291ae5924c215308805.jpg@160w_220h_1e_1c', 'title': '神偷奶爸', 'actor': '史蒂夫·卡瑞尔,杰森·席格尔,拉塞尔·布兰德', 'time': '20

{'index': '95', 'image': 'http://p1.meituan.net/movie/36a893c53a13f9bb934071b86ae3b5c492427.jpg@160w_220h_1e_1c', 'title': '爱·回家', 'actor': '俞承豪,金艺芬,童孝熙', 'time': '2002-04-05(韩国)', 'score': '9.0'}
{'index': '96', 'image': 'http://p1.meituan.net/movie/9bff56ed3ea38bb1825daa1d354bc92352781.jpg@160w_220h_1e_1c', 'title': '黄金三镖客', 'actor': '克林特·伊斯特伍德,李·范·克里夫,埃里·瓦拉赫', 'time': '1966-12-23(意大利)', 'score': '8.9'}
{'index': '97', 'image': 'http://p1.meituan.net/movie/ed50b58bf636d207c56989872a91f4cf305138.jpg@160w_220h_1e_1c', 'title': '我爱你', 'actor': '宋在浩,李顺才,尹秀晶', 'time': '2011-02-17(韩国)', 'score': '9.0'}
{'index': '98', 'image': 'http://p1.meituan.net/movie/a1634f4e49c8517ae0a3e4adcac6b0dc43994.jpg@160w_220h_1e_1c', 'title': '迁徙的鸟', 'actor': '雅克·贝汉,Philippe Labro', 'time': '2001-12-12(法国)', 'score': '9.1'}
{'index': '99', 'image': 'http://p0.meituan.net/movie/885fc379c614a2b4175587b95ac98eb95045650.jpg@160w_220h_1e_1c', 'title': '阿飞正传', 'actor': '张国荣,张曼玉,刘德华', 'time': '2018-06-25', 'score': 